<a href="https://colab.research.google.com/github/Data-Creater-Atlas/Data-Atlas/blob/yejun/mission_2_0918.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install segmentation-models-pytorch -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 13.4 MB/s eta 0:00:00


In [3]:
import numpy as np # 선형대수
import pandas as pd # 데이터 처리, CSV 파일 입출력 (예: pd.read_csv)
import os
import glob
import torch
import torch.nn as nn
import torch.utils as utils
import torch.nn.init as init
import torch.utils.data as data
import torchvision.utils as v_utils
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np
from random import shuffle
import scipy.misc
from PIL import Image
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn
import cv2
import os
import glob

/tmp/ipython-input-620625467.py:16: DeprecationWarning: scipy.misc is deprecated and will be removed in 2.0.0
  import scipy.misc


In [4]:
''' 1. Module Import '''
import torch
import os
import matplotlib.pyplot as plt
from IPython.display import Image

# Google Drive에 프로젝트 폴더를 생성하고 경로를 지정합니다.
# 이렇게 하면 결과가 내 드라이브에 저장됩니다.
GDRIVE_PROJECT_PATH = "/content/my_mission2_data"
if not os.path.exists(GDRIVE_PROJECT_PATH):
    os.makedirs(GDRIVE_PROJECT_PATH)

In [5]:
''' 2. 딥러닝 모델을 설계할 때 활용하는 장비 확인 '''
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
    print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)
    print('CUDA is available. Using GPU for training.')
else:
    DEVICE = torch.device('cpu')
    print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)
    print('CUDA is not available. Using CPU for training.')

Using PyTorch version: 2.8.0+cu126  Device: cuda
CUDA is available. Using GPU for training.


In [6]:
DATA_ROOT = "/content/drive/MyDrive/mission2_dataset"

import os, pathlib
pathlib.Path(DATA_ROOT).mkdir(parents=True, exist_ok=True)
print("DATA_ROOT : ", DATA_ROOT)

DATA_ROOT :  /content/drive/MyDrive/mission2_dataset


In [7]:
from pathlib import Path

# 원본 데이터( 이미지 / 라벨 JSON )
TS_DIR = Path(DATA_ROOT) / "train"  / "TS_KS"
TL_JSON_DIR = Path(DATA_ROOT) / "train"  / "TL_KS_LINE"
VS_DIR = Path(DATA_ROOT) / "valid"  / "VS_KS"
VL_JSON_DIR = Path(DATA_ROOT) / "valid" / "VL_KS_LINE"

# RESNET 데이터셋 출력 위치
DATASET_DIR = Path(DATA_ROOT) / "RESNET_Dataset"
(TR_IMG_DIR := DATASET_DIR/"train/images").mkdir(parents=True, exist_ok=True)
(TR_LABEL_DIR := DATASET_DIR/"train/labels").mkdir(parents=True, exist_ok=True)
(VAL_IMG_DIR := DATASET_DIR/"valid/images").mkdir(parents=True, exist_ok=True)
(VAL_LABEL_DIR := DATASET_DIR/"valid/labels").mkdir(parents=True, exist_ok=True)

# 학습에 사용할 CSV 출력 경로 (이미지 경로 + height_m)
TRAIN_CSV = DATASET_DIR / "train.csv"
VAL_CSV   = DATASET_DIR / "valid.csv"

In [8]:
# === GSD: JSON에 m_per_px 없을 때 fallback (Kompsat-3/3A, 1m 이하) ===
# Kompsat-3  ≈ 0.70 m/px, Kompsat-3A ≈ 0.55 m/px
SATELLITE = 'K3A'  # 'K3' or 'K3A'
DEFAULT_MPP_MAP = {'K3': 0.70, 'K3A': 0.55}
GLOBAL_MPP_FALLBACK = DEFAULT_MPP_MAP[SATELLITE]

In [9]:
# === 학습 하이퍼파라미터(512 타일에 맞춤) ===
IMG_SIZE      = 512   # 데이터가 512x512이므로 그대로 사용
BATCH_SIZE    = 32
EPOCHS        = 30
LR            = 1e-4
WEIGHT_DECAY  = 1e-4

In [10]:
# 체크포인트 / 예측 결과
CHECKPOINT = DATASET_DIR / "resnet18_height_best.pt"
PRED_CSV   = DATASET_DIR / "predictions_valid.csv"

In [11]:
import os, json, math, shutil
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm

IMAGE_EXTS = (".jpg",".jpeg",".png",".tif",".tiff",".bmp")

def _line_length_px(sa: dict):
    """VIA shape_attributes에서 선 길이(px) 계산: line 또는 polyline 지원"""
    n = sa.get("name")
    if n == "line":
        x1,y1,x2,y2 = sa.get("x1"),sa.get("y1"),sa.get("x2"),sa.get("y2")
        if None in [x1,y1,x2,y2]: return None
        return float(math.hypot(x2-x1, y2-y1))
    if n == "polyline":
        xs, ys = sa.get("all_points_x", []), sa.get("all_points_y", [])
        if not xs or not ys or len(xs) != len(ys): return None
        return float(sum(math.hypot(xs[i]-xs[i-1], ys[i]-ys[i-1]) for i in range(1, len(xs))))
    return None  # rect 등 다른 타입은 높이 회귀에 불필요

def _mpp_from_attrs(attrs: dict):
    """region_attributes에서 m/px를 찾는다(여러 키 허용). 찾지 못하면 None."""
    if not isinstance(attrs, dict): return None
    for k in ["m_per_px","meters_per_pixel","meter_per_pixel","mpp","scale"]:
        if k in attrs:
            try: return float(attrs[k])
            except: pass
    return None

def _scan_all_json_mpp(json_dir: Path):
    """폴더 안 모든 VIA JSON에서 m_per_px 값들을 수집"""
    mpps=[]
    for jp in sorted(json_dir.glob("*.json")):
        try:
            with open(jp, "r", encoding="utf-8") as f:
                data=json.load(f)
        except Exception as e:
            print(f"[WARN] JSON read fail: {jp} ({e})"); continue
        items = data.values() if isinstance(data, dict) else data
        for it in items:
            for r in it.get("regions", []):
                mpp = _mpp_from_attrs(r.get("region_attributes", {}))
                if mpp: mpps.append(mpp)
    return mpps

def _copy_and_label_from_json_dir(
    images_dir: Path, json_dir: Path, out_img_dir: Path, out_lbl_dir: Path,
    default_mpp: float
):
    """
    json_dir 안의 모든 VIA JSON을 읽어서:
      - 이미지 파일을 out_img_dir로 복사
      - 이미지별 높이(m)를 계산해 out_lbl_dir/{stem}.txt 로 저장(옵션)
      - (filepath,height_m) 리스트를 반환 (filepath는 out_img_dir 내 복사본 경로)
    """
    rows = []
    out_img_dir.mkdir(parents=True, exist_ok=True)
    out_lbl_dir.mkdir(parents=True, exist_ok=True)

    for jp in tqdm(sorted(json_dir.glob("*.json")), desc=f"[parse] {json_dir.name}"):
        try:
            with open(jp, "r", encoding="utf-8") as f:
                data = json.load(f)
        except Exception as e:
            print(f"[WARN] skip {jp}: {e}")
            continue

        items = data.values() if isinstance(data, dict) else data
        temp = {}  # {img_name: [heights(m), ...]}

        for it in items:
            fn = it.get("filename") or it.get("file", {}).get("name")
            if not fn:
                continue

            # 원본 이미지 경로 찾기
            src = images_dir / fn
            if not (str(src).lower().endswith(IMAGE_EXTS) and src.exists()):
                # 확장자 다를 수 있으니 stem 기반 후보 탐색
                cands = list(images_dir.glob(Path(fn).stem + ".*"))
                if cands:
                    src = cands[0]
                else:
                    # 이미지 못 찾으면 라벨 스킵
                    continue

            # 각 region의 길이(px) * m/px → m
            for r in it.get("regions", []):
                length = _line_length_px(r.get("shape_attributes", {}))
                if not length:
                    continue
                mpp = _mpp_from_attrs(r.get("region_attributes", {})) or default_mpp
                if not mpp:
                    continue
                temp.setdefault(src, []).append(length * mpp)

        # 이미지 단위로 평균 높이 산출 + 복사 + .txt 저장
        for src, heights in temp.items():
            if not heights:
                continue
            height_m = float(np.mean(heights))

            dst = out_img_dir / src.name
            if not dst.exists():
                shutil.copy2(src, dst)

            # (옵션) 이미지별 단일 값 텍스트 라벨 저장 -> 디버깅/재사용에 유용
            (out_lbl_dir / f"{dst.stem}.txt").write_text(f"{height_m:.6f}\n", encoding="utf-8")

            rows.append({"filepath": str(dst), "height_m": height_m})

    return rows

# ===== 1) m_per_px 결정: JSON에 값이 있으면 중앙값, 없으면 위성 기본값 사용 =====
mpps_train = _scan_all_json_mpp(TL_JSON_DIR)
mpps_valid = _scan_all_json_mpp(VL_JSON_DIR)
all_mpps = mpps_train + mpps_valid
if all_mpps:
    MPP_USED = float(np.median(all_mpps))
    print(f"JSON 내 m_per_px 감지: n={len(all_mpps)}, median={MPP_USED:.6f} m/px")
else:
    MPP_USED = GLOBAL_MPP_FALLBACK
    print(f"JSON에 m_per_px 없음 → 기본값 사용: {MPP_USED:.2f} m/px (SAT={SATELLITE})")

# ===== 2) JSON 폴더 → RESNET_Dataset/* 로 이미지 복사 + 라벨 산출 =====
train_rows = _copy_and_label_from_json_dir(
    images_dir=TS_DIR, json_dir=TL_JSON_DIR,
    out_img_dir=TR_IMG_DIR, out_lbl_dir=TR_LABEL_DIR,
    default_mpp=MPP_USED
)
valid_rows = _copy_and_label_from_json_dir(
    images_dir=VS_DIR, json_dir=VL_JSON_DIR,
    out_img_dir=VAL_IMG_DIR, out_lbl_dir=VAL_LABEL_DIR,
    default_mpp=MPP_USED
)

# ===== 3) CSV 저장 (학습/검증 입력으로 사용) =====
pd.DataFrame(train_rows).to_csv(TRAIN_CSV, index=False)
pd.DataFrame(valid_rows).to_csv(VAL_CSV, index=False)
print(f"[done] train: {len(train_rows)}  valid: {len(valid_rows)}")
print(f"CSV saved → {TRAIN_CSV}")
print(f"CSV saved → {VAL_CSV}")


JSON에 m_per_px 없음 → 기본값 사용: 0.55 m/px (SAT=K3A)


[parse] VL_KS_LINE: 100%|██████████| 1006/1006 [11:18<00:00,  1.48it/s]


[done] train: 8052  valid: 1006
CSV saved → /content/drive/MyDrive/mission2_dataset/RESNET_Dataset/train.csv
CSV saved → /content/drive/MyDrive/mission2_dataset/RESNET_Dataset/valid.csv


# 3) Dataset/DataLoader & 변환 (CSV 사용)

In [12]:
import cv2, torch
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pandas as pd

# 증강: 512 타일이므로 과도한 변형은 피하고 약하게
train_tf = A.Compose([
    A.LongestMaxSize(IMG_SIZE),
    A.PadIfNeeded(IMG_SIZE, IMG_SIZE, border_mode=cv2.BORDER_REFLECT_101),
    A.ShiftScaleRotate(shift_limit=0.03, scale_limit=0.05, rotate_limit=10,
                       border_mode=cv2.BORDER_REFLECT_101, p=0.5),
    A.RandomBrightnessContrast(0.1, 0.1, p=0.4),
    A.Normalize(), ToTensorV2()
])
val_tf = A.Compose([
    A.LongestMaxSize(IMG_SIZE),
    A.PadIfNeeded(IMG_SIZE, IMG_SIZE, border_mode=cv2.BORDER_REFLECT_101),
    A.Normalize(), ToTensorV2()
])

class ChimneyDataset(torch.utils.data.Dataset):
    """CSV의 (filepath,height_m)를 읽어 회귀 학습에 사용"""
    def __init__(self, csv_path: Path, tfm):
        self.df = pd.read_csv(csv_path)
        self.tfm = tfm
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        row = self.df.iloc[i]
        img = cv2.imread(row.filepath, cv2.IMREAD_COLOR)
        if img is None:
            raise FileNotFoundError(row.filepath)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        x = self.tfm(image=img)["image"]
        y = torch.tensor(row.height_m, dtype=torch.float32)
        return x, y, row.filepath

train_loader = DataLoader(ChimneyDataset(TRAIN_CSV, train_tf),
                          batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader   = DataLoader(ChimneyDataset(VAL_CSV,   val_tf),
                          batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)


/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [13]:
import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights
import torch

class ResNetRegressor(nn.Module):
    """ImageNet 사전학습 ResNet18 → 회귀 헤드(출력 1: meters)"""
    def __init__(self, pretrained=True):
        super().__init__()
        base = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1 if pretrained else None)
        in_f = base.fc.in_features
        base.fc = nn.Sequential(
            nn.Linear(in_f, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(256, 1)
        )
        self.net = base
    def forward(self, x):
        return self.net(x).squeeze(1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNetRegressor(pretrained=True).to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 228MB/s]


In [15]:
from pathlib import Path

p = Path("/content/drive/MyDrive/mission2_dataset/RESNET_Dataset/train/images/K3A_CHN_20231028055053_61.jpg")
print("exists?", p.exists())


exists? True


In [16]:
import pandas as pd, os
from pathlib import Path

def find_missing_paths(csv_path):
    df = pd.read_csv(csv_path)
    miss = df[~df["filepath"].apply(lambda x: Path(x).exists())]
    print(f"{csv_path.name}: total={len(df)}, missing={len(miss)}")
    return df, miss

train_df, miss_train = find_missing_paths(TRAIN_CSV)
valid_df, miss_valid = find_missing_paths(VAL_CSV)

miss_train.head()


train.csv: total=8052, missing=0
valid.csv: total=1006, missing=0


,filepath,height_m


In [18]:
import shutil, glob
from pathlib import Path
import pandas as pd

def _find_source_by_stem(stem, images_dir: Path):
    # stem.* 를 모두 탐색 (대소문자/확장자 임의)
    pats = []
    for ext in ["jpg","jpeg","png","tif","tiff","bmp","JPG","JPEG","PNG","TIF","TIFF","BMP"]:
        pats += glob.glob(str(images_dir / f"{stem}.{ext}"))
    return Path(pats[0]) if pats else None

def heal_csv(csv_path: Path, images_dir_src: Path, images_dir_dst: Path):
    df = pd.read_csv(csv_path)
    fixed_rows = 0
    dropped_rows = 0
    for i,row in df.iterrows():
        p = Path(row.filepath)
        if p.exists():
            continue
        # 없으면 원본에서 stem으로 재검색
        src = _find_source_by_stem(p.stem, images_dir_src)
        if src and src.exists():
            dst = images_dir_dst / src.name
            dst.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(src, dst)
            df.at[i, "filepath"] = str(dst)
            fixed_rows += 1
        else:
            # 원본에도 없으면 제거
            df.at[i, "filepath"] = None
            dropped_rows += 1

    # None 행 제거
    df = df.dropna(subset=["filepath"]).reset_index(drop=True)
    df.to_csv(csv_path, index=False)
    print(f"[heal] {csv_path.name}: fixed={fixed_rows}, dropped={dropped_rows}, remain={len(df)}")
    return df

# train/valid 각각 치유 실행
train_df = heal_csv(TRAIN_CSV, TS_DIR, TR_IMG_DIR)
valid_df = heal_csv(VAL_CSV, VS_DIR, VAL_IMG_DIR)


[heal] train.csv: fixed=0, dropped=0, remain=8052
[heal] valid.csv: fixed=0, dropped=0, remain=1006


# 3) 보강된 Dataset: cv2→PIL→numpy 순서로 재시도 (+스킵 안전장치)

In [19]:
import cv2, torch, numpy as np
from PIL import Image
import pandas as pd
from torch.utils.data import Dataset

def robust_read_rgb(path: str):
    # 1) cv2
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    if img is not None:
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # 2) PIL
    try:
        with Image.open(path) as im:
            return np.array(im.convert("RGB"))
    except:
        # 3) numpy fromfile + cv2.imdecode (경우에 따라 도움이 됨)
        try:
            data = np.fromfile(path, dtype=np.uint8)
            img = cv2.imdecode(data, cv2.IMREAD_COLOR)
            if img is not None:
                return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        except:
            pass
    return None

class ChimneyDatasetSafe(Dataset):
    def __init__(self, csv_path: Path, tfm):
        self.df = pd.read_csv(csv_path)
        self.tfm = tfm
        # 존재 & 디코딩 가능 샘플만 남김
        mask = self.df["filepath"].apply(lambda p: Path(p).exists() and robust_read_rgb(p) is not None)
        if mask.sum() == 0:
            raise ValueError(f"No decodable images in {csv_path}")
        self.df = self.df[mask].reset_index(drop=True)

    def __len__(self): return len(self.df)

    def __getitem__(self, i):
        row = self.df.iloc[i]
        img = robust_read_rgb(row.filepath)
        if img is None:
            # 드물게 I/O 실패 시, 인접 샘플로 대체(학습 중단 방지)
            j = (i+1) % len(self.df)
            row = self.df.iloc[j]
            img = robust_read_rgb(row.filepath)
            if img is None:
                raise RuntimeError(f"Unreadable image even after retry: {row.filepath}")
        x = train_tf(image=img)['image'] if self.tfm_mode=='train' else val_tf(image=img)['image']
        y = torch.tensor(row.height_m, dtype=torch.float32)
        return x, y, row.filepath

    @property
    def tfm_mode(self):  # 간단 모드 스위치
        return 'train' if self.tfm is train_tf else 'val'

#

#Robust Dataset (cv2 → PIL → imdecode 순서) + 안전 필터링

In [22]:
# --- Robust image reader: cv2 실패 시 PIL로 자동 폴백, 그래도 안되면 imdecode 시도 ---
import cv2, numpy as np
from PIL import Image
from pathlib import Path

def robust_read_rgb(path: str):
    # 1) cv2
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    if img is not None:
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 2) PIL
    try:
        with Image.open(path) as im:
            return np.array(im.convert("RGB"))
    except Exception:
        pass

    # 3) numpy fromfile + cv2.imdecode (일부 특이 JPEG에 효과)
    try:
        data = np.fromfile(path, dtype=np.uint8)
        img = cv2.imdecode(data, cv2.IMREAD_COLOR)
        if img is not None:
            return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    except Exception:
        pass

    return None


In [24]:
# --- Dataset: 존재 & 디코딩 가능 샘플만 유지, __getitem__에서도 폴백 재시도 ---
import pandas as pd
import torch
from torch.utils.data import Dataset

class ChimneyDatasetSafe(Dataset):
    def __init__(self, csv_path: Path, tfm):
        self.df = pd.read_csv(csv_path)
        self.tfm = tfm

        # 사전 필터링: 파일 존재 + 디코딩 가능 여부 검사 (빠른 스캔)
        mask = self.df["filepath"].apply(lambda p: Path(p).exists() and (robust_read_rgb(p) is not None))
        dropped = (~mask).sum()
        if dropped > 0:
            print(f"[ChimneyDatasetSafe] drop {dropped} unreadable rows from {Path(csv_path).name}")
        self.df = self.df[mask].reset_index(drop=True)

        if len(self.df) == 0:
            raise ValueError(f"No decodable samples remain in {csv_path}")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        row = self.df.iloc[i]
        img = robust_read_rgb(row.filepath)

        # 드물게 I/O 일시 오류 시 한 번 더 재시도(인접 샘플로 대체)
        if img is None:
            j = (i + 1) % len(self.df)
            row = self.df.iloc[j]
            img = robust_read_rgb(row.filepath)
            if img is None:
                # 정말 안 되면 해당 샘플을 건너뛰도록 예외
                raise RuntimeError(f"Unreadable image even after retry: {row.filepath}")

        x = self.tfm(image=img)["image"]
        y = torch.tensor(row.height_m, dtype=torch.float32)
        return x, y, row.filepath


In [25]:
# --- DataLoader: 문제 추적을 위해 처음엔 num_workers=0으로 돌려보고 OK면 늘리기 ---
from torch.utils.data import DataLoader

train_loader = DataLoader(
    ChimneyDatasetSafe(TRAIN_CSV, train_tf),
    batch_size=BATCH_SIZE, shuffle=True,
    num_workers=0, pin_memory=True, persistent_workers=False
)

val_loader = DataLoader(
    ChimneyDatasetSafe(VAL_CSV, val_tf),
    batch_size=BATCH_SIZE, shuffle=False,
    num_workers=0, pin_memory=True, persistent_workers=False
)

print("train batches:", len(train_loader), " | val batches:", len(val_loader))


[ChimneyDatasetSafe] drop 1 unreadable rows from train.csv
train batches: 252  | val batches: 32


# 학습(RMSE 기준 Early-Stopping)

In [26]:
from tqdm.auto import tqdm
import numpy as np, math, torch, os

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion = nn.MSELoss()

def rmse_torch(pred, target):
    return torch.sqrt(torch.mean((pred - target)**2))

best_rmse = float("inf")
patience, pat = 7, 0

for ep in range(1, EPOCHS+1):
    # ---- train ----
    model.train(); tr_rmse_sum = 0.0
    for x, y, _ in tqdm(train_loader, desc=f"E{ep} train"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad(set_to_none=True)
        p = model(x)
        loss = criterion(p, y)
        loss.backward(); optimizer.step()
        tr_rmse_sum += rmse_torch(p.detach(), y).item() * x.size(0)
    tr_rmse = tr_rmse_sum / len(train_loader.dataset)

    # ---- valid ----
    model.eval(); va_rmse_sum = 0.0
    with torch.no_grad():
        for x, y, _ in tqdm(val_loader, desc=f"E{ep} valid"):
            x, y = x.to(device), y.to(device)
            p = model(x)
            va_rmse_sum += rmse_torch(p, y).item() * x.size(0)
    va_rmse = va_rmse_sum / len(val_loader.dataset)

    print(f">> E{ep}: train_RMSE={tr_rmse:.3f} | val_RMSE={va_rmse:.3f}")

    # ---- early stop on RMSE ----
    if va_rmse + 1e-4 < best_rmse:
        best_rmse, pat = va_rmse, 0
        torch.save({
            "model": model.state_dict(),
            "val_rmse": float(best_rmse),
            "epoch": ep,
            "m_per_px_used": float(MPP_USED)
        }, CHECKPOINT)
        print(f"   ✔ best updated & saved → {CHECKPOINT}")
    else:
        pat += 1
        if pat >= patience:
            print("   ⏹ early stopping")
            break


E1 train:   0%|          | 0/252 [00:00<?, ?it/s]

E1 valid:   0%|          | 0/32 [00:00<?, ?it/s]

>> E1: train_RMSE=10.972 | val_RMSE=9.638
   ✔ best updated & saved → /content/drive/MyDrive/mission2_dataset/RESNET_Dataset/resnet18_height_best.pt


E2 train:   0%|          | 0/252 [00:00<?, ?it/s]

E2 valid:   0%|          | 0/32 [00:00<?, ?it/s]

...

E21 valid:   0%|          | 0/32 [00:00<?, ?it/s]

>> E21: train_RMSE=4.793 | val_RMSE=6.126


E22 train:   0%|          | 0/252 [00:00<?, ?it/s]

E22 valid:   0%|          | 0/32 [00:00<?, ?it/s]

>> E22: train_RMSE=4.725 | val_RMSE=6.096
   ⏹ early stopping


# 검증 예측 저장 & 최종 RMSE

In [27]:
import math, pandas as pd, torch

ckpt = torch.load(CHECKPOINT, map_location=device)
model.load_state_dict(ckpt["model"]); model.eval()

rows=[]
with torch.no_grad():
    for x, y, paths in tqdm(val_loader, desc="Predict (valid)"):
        x = x.to(device)
        pr = model(x).cpu().numpy()
        y = y.cpu().numpy()
        for pth, gt, pv in zip(paths, y, pr):
            rows.append({"filepath": pth, "height_gt_m": float(gt), "height_pred_m": float(pv)})

pred_df = pd.DataFrame(rows)
rmse = math.sqrt(np.mean((pred_df["height_pred_m"] - pred_df["height_gt_m"])**2))
print("Validation RMSE (m):", round(rmse, 3))

pred_df.to_csv(PRED_CSV, index=False)
print("Saved:", PRED_CSV)
pred_df.head(10)


Predict (valid):   0%|          | 0/32 [00:00<?, ?it/s]

Validation RMSE (m): 5.725
Saved: /content/drive/MyDrive/mission2_dataset/RESNET_Dataset/predictions_valid.csv


,filepath,height_gt_m,height_pred_m
0,/content/drive/MyDrive/mission2_dataset/RESNET...,75.315865,78.519180
1,/content/drive/MyDrive/mission2_dataset/RESNET...,37.886177,40.005260
2,/content/drive/MyDrive/mission2_dataset/RESNET...,30.109674,31.289967
3,/content/drive/MyDrive/mission2_dataset/RESNET...,27.941948,18.880739
4,/content/drive/MyDrive/mission2_dataset/RESNET...,30.337433,33.213863
5,/content/drive/MyDrive/mission2_dataset/RESNET...,16.086634,14.740008
6,/content/drive/MyDrive/mission2_dataset/RESNET...,31.747913,35.545135
7,/content/drive/MyDrive/mission2_dataset/RESNET...,32.417355,31.482515
8,/content/drive/MyDrive/mission2_dataset/RESNET...,27.400822,20.413349
9,/content/drive/MyDrive/mission2_dataset/RESNET...,25.190088,26.993423
